In [ ]:
#For Colab
#!pip install --upgrade pip
#!pip install --upgrade google-api-python-client
#!pip install googletrans
#!pip install tqdm --upgrade
#!pip install twython

# web scraping
import requests
from bs4 import BeautifulSoup
from time import time, sleep
from random import randint

#Utilities
from tqdm import tqdm
tqdm.pandas()
import copy
import numpy as np
import pandas as pd


from googleapiclient.discovery import build 

In [ ]:
def honestdoc_hosp_name(url):
    """GET All hospital name from the Honestdoc Website"""
    #create connection
    data = requests.get(url)
    print("requests code : {}".format(data.status_code)) 
    print("note\n2xx: success\n4xx, 5xx: error")
    
    #scrape comment and score
    start_time = time() #start scraping data from page1
    r = requests.get(url, params=dict(query="web scraping",page=1)) 
    soup = BeautifulSoup(r.text,"html.parser")
    
    h = len(soup.find_all("a",{"class":"preview__text preview__text--big"})) #find h hospitals in page
    ref = len(soup.find_all("a", {"class":"docket__link"}, href=True))
    #extract each item
   
    hospital = [soup.find_all("a",
                              {"class":"preview__text preview__text--big"})[j].get_text().strip() for j in range(0,h)]
    link = [soup.find_all("a",
                              {"class":"docket__link"}, href=True)[i]['href'] for i in range(0,ref)]
    p_hosp = 2

    while h > 0:
      start_time = time()
      r = requests.get(url, params=dict(query="web scraping",page=p_hosp)) 
      soup = BeautifulSoup(r.text,"html.parser")
    
      h = len(soup.find_all("a",{"class":"preview__text preview__text--big"})) #find h hospitals in page 
      ref = len(soup.find_all("a", {"class":"docket__link"}, href=True))
      [hospital.append(soup.find_all("a",
                                    {"class":"preview__text preview__text--big"})[j].get_text().strip()) for j in range(0,h)]
      [link.append(soup.find_all("a",
                              {"class":"docket__link"}, href=True)[i]['href']) for i in range(0,ref)]
      elapsed_time = time() - start_time
      print("Time used for scraping data from page - {} : {} s".format(p_hosp, elapsed_time))
      p_hosp +=1
      sleep(randint(1,3))
   
    pd.DataFrame({"hospital": hospital,
                  "link": link }).to_csv(PATH + "hospital_name.csv", index=False)
    
    return hospital, link

In [ ]:
def scrape_all_honestdoc_comment(urls):
    """
    This function is to scrap data from a webboard (https://www.honestdocs.com).

    """
    web = "https://www.honestdocs.co"
    url_names = urls.link
    hosp_names = urls.hospital
    df = pd.DataFrame(columns=["comment","score","hospital"])
    for hosp_name,url in zip(hosp_names,url_names):
        #create connection
        data = requests.get(web+url)
        print("Scrape from {}".format(hosp_name))
        print("requests code : {}".format(data.status_code))     
        
        #scrape comment and score
        try:
            start_time = time() #start scraping data from page1
            r = requests.get(web+url, params=dict(query="web scraping",page=1), allow_redirects=False) 
            soup = BeautifulSoup(r.text,"html.parser")
            n = len(soup.find_all("div",{"class":"comments__content"})) #find n of items in the page
            while n > 0:
              #extract each item
              comment = [soup.find_all("div",
                                      {"class":"comments__content"})[i].get_text().strip() for i in range(0,n)]
              score = [soup.find_all("span",
                                    {"class":"stars star-rating"})[i].attrs["data-score"] for i in range(0,n)]
                    
              p = 2 #start scraping data from page2
              while n > 0: #until the number of items in a page = 0
                  
                  r = requests.get(web+url, params=dict(query="web scraping",page=p), allow_redirects=False)
                  soup = BeautifulSoup(r.text,"html.parser")
                  n = len(soup.find_all("div",{"class":"comments__content"}))
                  [comment.append(soup.find_all("div",
                                                {"class":"comments__content"})[i].get_text().strip()) for i in range(0,n)]
                  [score.append(soup.find_all("span",
                                              {"class":"stars star-rating"})[i].attrs["data-score"]) for i in range(0,n)]
                  p +=1
                  df_new = pd.DataFrame({"comment": comment, 
                        "score": score})
              df_new["hospital"] = hosp_name
              df = pd.concat([df,df_new], ignore_index=True)
              print("numbers of comments : {}".format(len(df)))
              elapsed_time = time() - start_time
              print("Time used for scraping data from {}: {} s".format(hosp_name, elapsed_time))
        except requests.exceptions.Timeout:
            pass  
        except requests.exceptions.TooManyRedirects:
            pass
        except requests.exceptions.RequestException as e:
    # catastrophic error. bail.
            raise SystemExit(e)
    df.to_csv("all_honestdoc_comments.csv", index=False)
    return df

In [ ]:
hosp, link = honestdoc_hosp_name("https://www.honestdocs.co/hospitals/")

requests code : 200
note
2xx: success
4xx, 5xx: error
Time used for scraping data from page - 2 : 1.4908597469329834 s
Time used for scraping data from page - 3 : 1.663114309310913 s
Time used for scraping data from page - 4 : 1.435826063156128 s
Time used for scraping data from page - 5 : 1.4327340126037598 s
Time used for scraping data from page - 6 : 1.6354801654815674 s
Time used for scraping data from page - 7 : 1.6256332397460938 s
Time used for scraping data from page - 8 : 1.743257999420166 s
Time used for scraping data from page - 9 : 1.6277379989624023 s
Time used for scraping data from page - 10 : 1.6306161880493164 s
Time used for scraping data from page - 11 : 1.6297845840454102 s
Time used for scraping data from page - 12 : 1.63319730758667 s
Time used for scraping data from page - 13 : 1.6467697620391846 s
Time used for scraping data from page - 14 : 1.6351346969604492 s
Time used for scraping data from page - 15 : 1.638066291809082 s
Time used for scraping data from pag

In [ ]:
urls = pd.DataFrame({'hosp':hosp,'link':link})

In [ ]:
df = scrape_all_honestdoc_comment(urls)

## Check the Duplicated comment

In [ ]:
df[df.duplicated('comment')].groupby('hospital').sum().head(50)

,comment,score
hospital,,
มิตรไมตรีคลินิกเวชกรรม สาขาบึงคำพร้อย,ดีมากๆครับ มีมาตรฐาน ความเรียบร้อยและดูแลคนไข้...,54
มิตรไมตรีคลินิกเวชกรรม สาขาพรประภานิมิตร,บริการดีมากครับ ใจดีและพดตรงดีครับเพราะสำคัญมา...,4
มิตรไมตรีคลินิกเวชกรรม สาขาวัดพระเงิน,ดีค่ะ ให้คำปรึกษาดี และรู้สึกอุ่นใจขึ้นเมื่อได...,55555444545544545555455445445555
โรงบาลเมโย,จากประสบการณ์ที่ได้ไปใช้บริการโรงพยาบาลเมโยมาน...,54
โรงพยาบาลกรุงเทพ,บริการดีประทับใจ,5
โรงพยาบาลกรุงเทพคริสเตียน,บริการดีมาก,4
โรงพยาบาลกรุงเทพคริสเตียน นครปฐม,บริการดีมาก,5
โรงพยาบาลกรุงเทพพิษณุโลก,โรงพยาบาลนี้ดีมากครับ รักษามาตรฐานความเป็นโรงพ...,5
โรงพยาบาลกรุงเทพภูเก็ต,ดีมากๆ,4


In [ ]:
df.loc[df.hospital.str.contains("มิตรไมตรีคลินิกเวชกรรม สาขาวัดพระเงิน")].head(50)

,comment,score,hospital
9343,ดูแลดีมากครับเป็นกันเอง หมอมีความใจดีเป็นกันเอ...,4,มิตรไมตรีคลินิกเวชกรรม สาขาวัดพระเงิน
9344,ดีค่ะ ให้คำปรึกษาดี และรู้สึกอุ่นใจขึ้นเมื่อได...,5,มิตรไมตรีคลินิกเวชกรรม สาขาวัดพระเงิน
9345,ดูแลผู้ป่วยดีครับ และ ให้คำแนะนำดีทุกๆอย่างไม่...,5,มิตรไมตรีคลินิกเวชกรรม สาขาวัดพระเงิน
9346,ดีมากๆเลยครับสะอาดตอนรับดีคุณหมอก้คุยเข้าใจง่า...,5,มิตรไมตรีคลินิกเวชกรรม สาขาวัดพระเงิน
9347,ดีค่ะ ให้คำปรึกษาดี และรู้สึกอุ่นใจขึ้นเมื่อได...,5,มิตรไมตรีคลินิกเวชกรรม สาขาวัดพระเงิน
9348,เข้าไปครั้งแรก รู้สึกได้ถึงความใส่ใจในการบริกา...,5,มิตรไมตรีคลินิกเวชกรรม สาขาวัดพระเงิน
9349,เข้าไปครั้งแรก รู้สึกได้ถึงความใส่ใจในการบริกา...,5,มิตรไมตรีคลินิกเวชกรรม สาขาวัดพระเงิน
9350,รักษาดีมากครับ หมอคุยดีมาก เป้นกันเองมห้กำลังใ...,5,มิตรไมตรีคลินิกเวชกรรม สาขาวัดพระเงิน
9351,เป็นโรงพยาบาลที่ดีมากดูแลใส่ใจสุขภาพดี ไม่ต้อง...,4,มิตรไมตรีคลินิกเวชกรรม สาขาวัดพระเงิน
9352,เป็นการรักษาและปรึกษาที่ดีครับ เท่าที่ไปมาเพรา...,5,มิตรไมตรีคลินิกเวชกรรม สาขาวัดพระเงิน


Drop the duplicated comment

In [ ]:
df.drop_duplicates(subset="comment",keep='first',inplace=True )
df.reset_index(inplace=True)

In [ ]:
df.duplicated().sum()

0

Translate Thai to English with Google Cloud Platform API

NLP Google Translate with GCP

In [ ]:
import getpass

APIKEY = getpass.getpass()

··········


In [ ]:
def th2en(df):   

  translated = []
  for index, row in df.iterrows():
    if index%50 == 0:
        print(index)
        sleep(15) #Due to GCP limitted characters per sec

    inputs = copy.deepcopy(row)
    service = build('translate', 'v2', developerKey=APIKEY)

    try:
        # translate the 'text' column
        outputs = service.translations().list(source='th', target='en', q=inputs['comment']).execute()

    except Exception as e:
        print(str(e))
        continue
    
    translated.append(outputs['translations'][0]['translatedText'])
  
  return pd.Series(translated)

In [ ]:
X = th2en(df)

In [ ]:
X.tail()

9142    Fast, very clean. The nurses are friendly. Bet...
9143    Good service, polite words, arrange things qui...
9144    Arrange things quickly. Go to wait in line. Wa...
9145    The doctor took very good care. The hospital i...
9146    The doctor took very good care. The hospital i...
dtype: object

In [ ]:
X.to_csv('translate.csv')

In [ ]:
df['en'] = X

Drop English duplicated

In [ ]:
df.drop_duplicates(subset="en",keep='first',inplace=True )
df.reset_index(inplace=True)

In [ ]:
#Save file
df.to_csv("honest_doc_full.csv", index=False)